# Import

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

# Hyperparameter

In [2]:
config = {
    "learning_rate": 5e-5,
    "epoch": 10,
    "batch_size": 128
}

CFG = SimpleNamespace(**config)

# Load Data

In [3]:
#augmented, 500개로 맞춤
#augmented2, 1000개로 맞춤
#augmented_combined, 동일 카테고리별로 5개씩 묶고, 합친 갯수가 200개보다 적으면 200개까지 증강
#augmented_combined3, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined4, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined5, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined6, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 7000개까지 줄임
#augmented_combined8, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 3000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 2000개까지 줄임
train_df = pd.read_csv("/kaggle/input/dacon-dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/dacon-dataset/test.csv")

In [4]:
train_df["분류"].value_counts()

분류
지역               26950
경제:부동산            3454
사회:사건_사고          2568
경제:반도체            2318
사회:사회일반           1480
사회:교육_시험           995
정치:국회_정당           966
사회:의료_건강           950
경제:취업_창업           845
스포츠:올림픽_아시안게임      841
경제:산업_기업           711
문화:전시_공연           671
경제:자동차             640
경제:경제일반            625
사회:장애인             621
스포츠:골프             617
정치:선거              608
경제:유통              589
IT_과학:모바일          537
사회:여성              536
사회:노동_복지           447
사회:환경              396
경제:서비스_쇼핑          387
경제:무역              375
정치:행정_자치           349
국제                 337
문화:방송_연예           335
스포츠:축구             328
경제:금융_재테크          327
정치:청와대             279
문화:출판              248
IT_과학:IT_과학일반      243
IT_과학:인터넷_SNS      238
문화:미술_건축           229
정치:정치일반            221
IT_과학:과학           215
문화:문화일반            213
문화:학술_문화재          202
문화:요리_여행           190
경제:자원              178
문화:종교              173
IT_과학:콘텐츠          160
사회:미디어             128
사회:날씨   

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# 1. '지역' 카테고리 필터링
region_df = train_df[train_df['분류'] == '지역'].copy()

# 2. 제목과 키워드 데이터를 각각 리스트로 변환
title_texts = region_df['제목'].tolist()
keyword_texts = region_df['키워드'].tolist()

# 3. TF-IDF 벡터화
title_vectorizer = TfidfVectorizer().fit_transform(title_texts)
keyword_vectorizer = TfidfVectorizer().fit_transform(keyword_texts)

# 4. 코사인 유사도 계산 (제목과 키워드 각각)
title_cosine_similarities = cosine_similarity(title_vectorizer, title_vectorizer)
keyword_cosine_similarities = cosine_similarity(keyword_vectorizer, keyword_vectorizer)

# 5. 유사도 임계값 설정 (0.8 이상이면 중복으로 간주)
threshold = 0.5

# 6. 중복 데이터 탐지 함수 (일부 제거)
def find_duplicate_indices(cosine_similarities, threshold):
    duplicate_indices = []
    for i in range(len(cosine_similarities)):
        for j in range(i + 1, len(cosine_similarities)):
            if cosine_similarities[i][j] > threshold:
                duplicate_indices.append(j)
    # 중복된 데이터 중에서 절반만 제거하도록 선택
    unique_duplicate_indices = list(set(duplicate_indices))
    to_remove = random.sample(unique_duplicate_indices, len(unique_duplicate_indices) // 2)
    return to_remove

# 7. 제목과 키워드에서 각각 유사한 데이터의 중복 인덱스 찾기
title_duplicate_indices = find_duplicate_indices(title_cosine_similarities, threshold)
keyword_duplicate_indices = find_duplicate_indices(keyword_cosine_similarities, threshold)

# 8. 제목과 키워드 중복 인덱스의 교집합만 제거
duplicate_indices = list(set(title_duplicate_indices) | set(keyword_duplicate_indices)) 
# 9. 중복 데이터 제거
region_df_cleaned = region_df.drop(region_df.index[duplicate_indices]).reset_index(drop=True)


# 10. 중복된 데이터 수 확인
print(f"지역 카테고리에서 중복된 데이터 {len(duplicate_indices)}개 제거 완료.")

지역 카테고리에서 중복된 데이터 11667개 제거 완료.


In [6]:
# 11. 원래 데이터프레임에서 지역 카테고리를 업데이트
train_df_updated = pd.concat([train_df[train_df['분류'] != '지역'], region_df_cleaned], ignore_index=True)

# 제거된 후의 데이터프레임 확인
print(f"전체 데이터 크기: {train_df_updated.shape}")

전체 데이터 크기: (42942, 4)


In [7]:
train_df_updated["분류"].value_counts()

분류
지역               15283
경제:부동산            3454
사회:사건_사고          2568
경제:반도체            2318
사회:사회일반           1480
사회:교육_시험           995
정치:국회_정당           966
사회:의료_건강           950
경제:취업_창업           845
스포츠:올림픽_아시안게임      841
경제:산업_기업           711
문화:전시_공연           671
경제:자동차             640
경제:경제일반            625
사회:장애인             621
스포츠:골프             617
정치:선거              608
경제:유통              589
IT_과학:모바일          537
사회:여성              536
사회:노동_복지           447
사회:환경              396
경제:서비스_쇼핑          387
경제:무역              375
정치:행정_자치           349
국제                 337
문화:방송_연예           335
스포츠:축구             328
경제:금융_재테크          327
정치:청와대             279
문화:출판              248
IT_과학:IT_과학일반      243
IT_과학:인터넷_SNS      238
문화:미술_건축           229
정치:정치일반            221
IT_과학:과학           215
문화:문화일반            213
문화:학술_문화재          202
문화:요리_여행           190
경제:자원              178
문화:종교              173
IT_과학:콘텐츠          160
사회:미디어             128
사회:날씨   

# Load Model

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
#tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Custom Dataset

In [9]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [10]:
# 데이터 준비
train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.제목_키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.제목_키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.제목_키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [11]:
test_df

,ID,제목,키워드,제목_키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대...","[부고] 김태수씨 별세 외 김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바...","신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력 신규,확진,나..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전...","[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의...","용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사 용인시,구인,장애인,구직..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경...","지자체 벽 터 경기지역 산단 활성화 모색 지자체,경기,북동부,지역,산업단지,혁신단위..."
...,...,...,...,...
23400,TEST_23400,코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속,"코로나19,감염,경로,조사,32.4%,최고,일상감염,지속,기준,확진자,기준,코로나1...","코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속 코로나19,감염,경..."
23401,TEST_23401,“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란,"여행,외식,자제,vs,모임,시민들,혼란,인천국제공항,아시아나항공,한반도,일주,비행,...","“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란 여행,외식,자..."
23402,TEST_23402,송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹,"임야,송철호,울산,시장,배우자,용인,매입,의혹,송철호,울산,시장,배우자,경기,용인,...","송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹 임야,송철호,울산,시장,배우자,..."
23403,TEST_23403,여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사,"여직원,자궁,모형,사진,홍보용,한의사,한의원,간호조무사,동의,자궁,모형,사진,한의사...","여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사 여직원,자궁,모형..."


In [12]:
train_df["분류"].value_counts()

분류
지역               21560
경제:부동산            2763
사회:사건_사고          2054
경제:반도체            1854
사회:사회일반           1184
사회:교육_시험           796
정치:국회_정당           773
사회:의료_건강           760
경제:취업_창업           676
스포츠:올림픽_아시안게임      673
경제:산업_기업           569
문화:전시_공연           537
경제:자동차             512
경제:경제일반            500
사회:장애인             497
스포츠:골프             494
정치:선거              486
경제:유통              471
IT_과학:모바일          430
사회:여성              429
사회:노동_복지           358
사회:환경              317
경제:서비스_쇼핑          310
경제:무역              300
정치:행정_자치           279
국제                 270
문화:방송_연예           268
경제:금융_재테크          262
스포츠:축구             262
정치:청와대             223
문화:출판              198
IT_과학:IT_과학일반      194
IT_과학:인터넷_SNS      190
문화:미술_건축           183
정치:정치일반            177
IT_과학:과학           172
문화:문화일반            170
문화:학술_문화재          162
문화:요리_여행           152
경제:자원              142
문화:종교              138
IT_과학:콘텐츠          128
사회:미디어             102
사회:날씨   

In [13]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# 학습
model.train()
#for epoch in range(CFG.epoch):
for epoch in range(15):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{15}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())
    
    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 86/86 [01:31<00:00,  1.07s/it]


Validation F1 Score: 0.08


Validating: 100%|██████████| 86/86 [01:31<00:00,  1.07s/it]


Validation F1 Score: 0.22


Validating: 100%|██████████| 86/86 [01:32<00:00,  1.07s/it]


...

Validation F1 Score: 0.47


Validating: 100%|██████████| 86/86 [01:32<00:00,  1.08s/it]


Validation F1 Score: 0.49


Validating: 100%|██████████| 86/86 [01:31<00:00,  1.07s/it]

Validation F1 Score: 0.49


# Inference

In [15]:
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # 모델을 통한 예측
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        # 예측 결과 저장
        test_predictions.extend(preds.cpu().tolist())

# test.csv 파일에서 'id' 값을 그대로 사용
test_df = pd.read_csv('/kaggle/input/dacon-dataset/test.csv')  # test.csv에 'id' 칼럼이 존재한다고 가정

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]



Testing: 100%|██████████| 183/183 [03:17<00:00,  1.08s/it]


In [16]:
# 예측 결과를 데이터프레임으로 저장
df_results = pd.DataFrame({
    'ID': test_df['ID'],             # test.csv의 'id' 값을 그대로 사용
    '분류': decoded_predictions  # 디코딩된 예측 라벨
})

# CSV 파일로 저장
df_results.to_csv('/kaggle/working/submission.csv', index=False)

print("예측 결과가 submission.csv 파일로 저장되었습니다.")

예측 결과가 submission.csv 파일로 저장되었습니다.


In [17]:
df=pd.read_csv('/kaggle/working/submission.csv')

In [18]:
df.head(50)

,ID,분류
0,TEST_00000,사회:의료_건강
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:정치일반
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역
5,TEST_00005,경제:반도체
6,TEST_00006,경제:경제일반
7,TEST_00007,국제
8,TEST_00008,사회:사건_사고
9,TEST_00009,지역
